
<div dir=rtl >
<h1>قوانین انجمنی</h1>

قوانین انجمنی روابط و وابستگی‌های متقابل بین مجموعه بزرگی از اقلام داده‌ای را نشان می‌دهند. 

مثال متداول در رابطه با کشف قوانین انجمنی "تحلیل سبد خرید" است.  در این فرایند با توجه به اقلام مختلفی که مشتریان در سبد خریدشان قرار می‌دهند، عادات و رفتار خرید مشتریان مورد تحلیل قرار می‌گیرد و می‌توان با شناسایی ارتباط بین محصولات، الگوهای تکرار شونده در حین خرید را بدست آورد.

سه پارامتر مهم:
- Support  میزان محبوبیت یک مجموعه آیتم را با توجه به دفعات حضور در تراکنش‌ها نشان می‌دهد.
- Confidence  احتمال خرید آیتم y در صورت خرید آیتم x را نشان می‌دهد. x -> y
- Lift   ترکیب دو پارامتر بالا می‌باشد.

برای پیاده سازی قوانین انجمنی در این تمرین، از الگوریتم Apriori که یکی از محبوب‌ترین و کارآمدترین الگوریتم‌های این حوزه است استفاده می‌کنیم.


<font color='Green'> پرسش:  تاثیر مقادیر مختلف پارامتر Lift را در احتمال وقوع نتیجه بررسی کنید.  </font>

<font color='white'>جواب: پارامتر lift میزان همبستگی دو آیتم را با یکدیگر نشان می‌دهد بدین صورت که اگر مقدار lift زیاد باشد یعنی دو آیتم با یکدیگر همبستگی بالایی دارند و برعکس. در واقع این پارامتر، نسبت دفعاتی می‌باشد که دو آیتم با یکدیگر خریداری شده‌اند به تعداد دفعاتی که انتظار می‌رود این دو آیتم با یکدیگر خریداری شوند می‌باشد.  </font>

</div>





<div dir=rtl >
<h1>الگوریتم Apriori</h1>

روش کار الگوریتم به این صورت است که یک  حداقل مقدار پشتیبانی در نظر گرفته    می‌شود و تکرارها با مجموعه آیتم‌های مکرر اتفاق می‌افتند. در صورتیکه  مجموعه‌ها و زیرمجموعه‌ها مقدار پشتیبانی کمتر از حد آستانه داشته باشند حذف می‌شوند. این روند تا زمانیکه امکان حذف وجود نداشته باشد ادامه می‌یابد. 

در این بخش از تمرین می‌خواهیم الگوریتم apriori را بر روی مجموعه داده Hypermarket_dataset که شامل سفارشات خرید افراد از فروشگاه های مواد غذایی است اعمال کنیم.


</div>





<div dir=rtl >
<h1>آماده‌سازی داده</h1>

<font color='Green'>پرسش: برای شروع کار باید داده‌های دیتاست را در قالب ماتریس پراکنده که محصولات  خریداری شده در ستون‌ و شماره سفارشات خرید به عنوان شاخص (index) هستند آماده کنید. 


برای راحتی کار محصولات خریداری شده در هر سفارش را با اعداد 0,1 کدگذاری کنید.

 نمونه ماتریس خروجی شما:

<img src="https://drive.google.com/uc?id=1eD0jan1ZbeYqSklgK--ks7oeY-MyTA3p"></img>

</div>




In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

dataset = pd.read_csv('/content/gdrive/MyDrive/Uni./Data Mining/Hypermarket_dataset.csv')

Mounted at /content/gdrive


In [3]:
dataset.loc[dataset['Member_number'] == 1905]

,Member_number,Date,itemDescription
19,1905,07-07-2015,other vegetables
35,1905,21-02-2015,fruit/vegetable juice
875,1905,19-05-2015,sugar
5244,1905,18-10-2015,soda
6713,1905,31-07-2015,ham
7096,1905,07-07-2015,tropical fruit
7112,1905,21-02-2015,sausage
13447,1905,22-04-2014,photo/film
14090,1905,02-06-2014,cat food
15221,1905,31-01-2014,seasonal products


In [10]:
def items_to_array(items_array: list, purchased_items: list):
  vector = [0] * 167
  for item in purchased_items:
    item_index = items_array.index(item)
    vector[item_index] = 1
  return vector

In [7]:
dataset.loc[1]['itemDescription']

'whole milk'

In [12]:
# your code here
items = []
customer_IDs = []
member_transactions = {}

for i in range(len(dataset)):
  customer_id = dataset.loc[i]['Member_number']
  # items_tmp = dataset.loc[i]['itemDescription'].split('/')
  items_tmp = dataset.loc[i]['itemDescription']

  if customer_id not in customer_IDs:
    member_transactions[customer_id] = [items_tmp]
    customer_IDs.append(customer_id)
  else: 
    member_transactions[customer_id] += [items_tmp]

  # for item in items_tmp:
  #   if item not in items:
  #     items.append(item)

  if items_tmp not in items:
    items.append(items_tmp)

trasactions_vector = {}
for i in list(member_transactions.keys()):
  trasactions_vector[i] = items_to_array(items, list(set(member_transactions[i])))

In [13]:
transactions_df = pd.DataFrame.from_dict(trasactions_vector, orient='index', columns=items)
transactions_df

,tropical fruit,whole milk,pip fruit,other vegetables,rolls/buns,pot plants,citrus fruit,beef,frankfurter,chicken,...,flower (seeds),rice,tea,salad dressing,specialty vegetables,pudding powder,ready soups,make up remover,toilet cleaner,preservation products
1808,1,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2552,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2300,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1187,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3037,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4590,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4703,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



<div dir=rtl >
<h1>شناسایی الگوهای پر تکرار</h1>

<font color='Green'> پرسش:  با اعمال الگوریتم apriori و به ازای مقدار min_support = 0.07 تمامی الگوهای پر تکرار را تولید نمایید. 

</div>




In [14]:
# your code here
frequent_itemsets = apriori(transactions_df, min_support=0.07)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.233710,(0),1
1,0.458184,(1),1
2,0.170600,(2),1
3,0.376603,(3),1
4,0.349666,(4),1
...,...,...,...
78,0.097486,"(17, 37)",2
79,0.077219,"(18, 37)",2
80,0.081324,"(37, 21)",2
81,0.082093,"(1, 3, 4)",3



<div dir=rtl >
<h1>استخراج قوانین انجمنی</h1>


<font color='Green'> پرسش: تابعی بنویسید که با گرفتن دو ورودی confidence و lift قوانین انجمنی حاصل را در خروجی نشان دهد. </br>
برای دو حالت خروجی خود را در گزارش ثبت کنید.


</div>




In [15]:
# your code here
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
13,(17),"(1, 3)",0.282966,0.191380,0.071832,0.253853,1.326434,0.017678,1.083727,1,2
8,"(1, 3)",(17),0.191380,0.282966,0.071832,0.375335,1.326434,0.017678,1.147870,2,1
11,(1),"(17, 3)",0.458184,0.120318,0.071832,0.156775,1.303003,0.016704,1.043235,1,2
10,"(17, 3)",(1),0.120318,0.458184,0.071832,0.597015,1.303003,0.016704,1.344507,2,1
1,(18),(17),0.206003,0.282966,0.075423,0.366127,1.293892,0.017132,1.131196,1,1
0,(17),(18),0.282966,0.206003,0.075423,0.266546,1.293892,0.017132,1.082545,1,1
12,(3),"(1, 17)",0.376603,0.150590,0.071832,0.190736,1.266589,0.015119,1.049608,1,2
9,"(1, 17)",(3),0.150590,0.376603,0.071832,0.477002,1.266589,0.015119,1.191967,2,1
2,"(1, 3)",(4),0.191380,0.349666,0.082093,0.428954,1.226753,0.015174,1.138847,2,1
7,(4),"(1, 3)",0.349666,0.191380,0.082093,0.234776,1.226753,0.015174,1.056710,1,2


In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.07)
rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values("lift",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_length,consequents_length
96,(12),(13),0.124936,0.124936,0.124936,1.000000,8.004107,0.109327,inf,1,1
97,(13),(12),0.124936,0.124936,0.124936,1.000000,8.004107,0.109327,inf,1,1
111,(79),(78),0.154695,0.154695,0.154695,1.000000,6.464345,0.130764,inf,1,1
110,(78),(79),0.154695,0.154695,0.154695,1.000000,6.464345,0.130764,inf,1,1
142,"(1, 78)",(79),0.079785,0.154695,0.079785,1.000000,6.464345,0.067442,inf,2,1
...,...,...,...,...,...,...,...,...,...,...,...
34,(1),(41),0.458184,0.313494,0.151103,0.329787,1.051973,0.007465,1.024310,1,1
66,(41),(3),0.313494,0.376603,0.124166,0.396072,1.051695,0.006103,1.032237,1,1
67,(3),(41),0.376603,0.313494,0.124166,0.329700,1.051695,0.006103,1.024178,1,1
3,(3),(0),0.376603,0.233710,0.091329,0.242507,1.037642,0.003313,1.011614,1,1
